### Permutationen

Eine Permutation der Länge N ist eine Liste von 0,N-1 in anderer Reihenfolge. Ist Liste b aus a durch die Permutation p hervorgegangen, 
dann gilt:

```
b = [a[p[i]] for i in range(N)]

```

Wenn die Ausgangsliste a die sortierte Liste von 0 bis N-1 ist, dann ist die Permutation p gleich der Zielliste b.

In [230]:
import doctest

In [59]:
a = list('BCADE')
b = list('BDAEC')

In [305]:
def getPermutation(a, b):
    '''
    a, b: Listen 
    returns: Permutation, durch die b aus a hervorgeht

    >>> a = [2,5,4,3,1]
    >>> b = [3,1,5,4,2]
    >>> p = getPermutation(a, b)
    >>> p
    [3, 4, 1, 2, 0]
    >>> b == [a[p[i]] for i in range(len(a))]
    True
    '''
    tmp = []
    for x in b:
        tmp.append(a.index(x))
    return tmp

def applyPermutation(p, a):
    return [a[p[i]] for i in range(len(a))]

import doctest
doctest.run_docstring_examples(getPermutation,globals(),optionflags=doctest.NORMALIZE_WHITESPACE)

#### Permutation wiederholen

Permutation wiederholen mit einer Schleife

In [304]:


def repeatPermutation(p, k):
    '''
    p eine Permutation
    returns: Permutation nach k Wiederholungen
    Für sehr große Zahlen von k ungeeignet 

    >>> p = [2,0,4,1,3]   
    >>> repeatPermutation(p,0)
    [0, 1, 2, 3, 4]
    >>> repeatPermutation(p,1)
    [2, 0, 4, 1, 3]
    >>> repeatPermutation(p,2)
    [4, 2, 3, 0, 1]
    '''
    p1 = list(range(len(p)))
    for _ in range(k):
        p1 = [p1[p[i]] for i in range(len(p))]
    return p1 

import doctest
doctest.run_docstring_examples(repeatPermutation,globals(),optionflags=doctest.NORMALIZE_WHITESPACE)

Permutation wiederholen durch finden eines Zyklus


In [292]:
def getPermutationCycle(p):
    '''
    berechnet, nach wieviel Wiederholungen die Permutation p die identischen Permutation wird.
    >>> p = [2,0,4,1,3] 
    >>> getPermutationCycle(p)
    5
    >>> repeatPermutation(p,5)  
    [0, 1, 2, 3, 4]
    '''
    anz = 0
    p1 = p.copy()
    while True:
        p1 = [p1[p[i]] for i in range(len(p))]
        anz+=1
        if p1 == p:
            return anz

def repeatPermutationALot(p,k):
    '''
    Dies können wir anwenden, wenn wir vermuten, dass Wiederholungen der Permutation nicht zu lange benötigen,
    um zur identischen Permutation zu werden.
    '''
    k0 = getPermutationCycle(p)
    return repeatPermutation(p,k%k0)

import doctest
doctest.run_docstring_examples(getPermutationCycle,globals(),optionflags=doctest.NORMALIZE_WHITESPACE)

Permutation wiederholen durch binary lifting (precompute die 2er Potenzen der Wiederholungen)

In [291]:
import math
def getPermutationMatrix(p, maxK):
    '''
    p: Permutation, maxK: maximale Anzahl der Permutationen
    returns: Matrix mit den Ergebnissen der Permutation in 2er-Potenzen

    >>> p = [2,0,4,1,3]
    >>> getPermutationMatrix(p,31)  # 31 = (11110) binär
    [[2, 0, 4, 1, 3],
     [4, 2, 3, 0, 1],
     [1, 3, 0, 4, 2],
     [3, 4, 1, 2, 0],
     [2, 0, 4, 1, 3]]
    '''
    logval = int(math.log(maxK, 2))
    m = [p]
    for _ in range(logval):
        m.append([m[-1][m[-1][j]] for j in range(len(p))])
    return m

def repeatPermutationBinary(p, k, pMatrix):
    '''
    p: Permutation, k: Anzahl der Wiederholungen,
    pMatrix: Matrix mit den berechneten 2erPotenzen der Wiederholungen
    returns: Permutation nach k Wiederholungen

    >>> p = [2,0,4,1,3]
    >>> m = getPermutationMatrix(p,10**9)
    >>> repeatPermutationBinary(p, 12345678, m)
    [3, 4, 1, 2, 0]

    '''
    logval = int(math.log(k, 2))
    p = list(range(len(p)))
    for i in range(logval + 1):
        if k & (1 << i):
            p = [pMatrix[i][p[j]] for j in range(len(p))]
    return p

import doctest
doctest.run_docstring_examples(getPermutationMatrix,globals(),optionflags=doctest.NORMALIZE_WHITESPACE)
doctest.run_docstring_examples(repeatPermutationBinary,globals(),optionflags=doctest.NORMALIZE_WHITESPACE)

#### Wiederholte Permutationen anwenden

Die Permutation, die von a nach b führt, soll 20004 mal wiederholt werden:

In [ ]:
a = list('BCADEFG')
b = list('BDFAEGC')

In [311]:
def applyRepeatPermutation(a,b,k):
    p = getPermutation(a,b)
    pk = repeatPermutation(p,k)         # langsam für große k
    return applyPermutation(pk,a)
    
applyRepeatPermutation(a,b,20004)

['B', 'G', 'D', 'C', 'E', 'A', 'F']

In [312]:
def applyRepeatPermutation(a,b,k):
    p = getPermutation(a,b)
    pk = repeatPermutationALot(p,k)       # Schnell, wenn Zyklus schnell gefunden werden kann   
    return applyPermutation(pk,a)

applyRepeatPermutation(a,b,20004)

['B', 'G', 'D', 'C', 'E', 'A', 'F']

In [313]:
def applyRepeatPermutation(a,b,k,m):
    p = getPermutation(a,b)
    pk = repeatPermutationBinary(p,k,m)     # Schnell auch für sehr große Zahlen k 
    return applyPermutation(pk,a)

p = getPermutation(a,b)
m = getPermutationMatrix(p,20004)          
applyRepeatPermutation(a,b,20004,m)

['B', 'G', 'D', 'C', 'E', 'A', 'F']